In [162]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# Initialize WebDriver
driver = webdriver.Chrome()  # You should replace this with the appropriate web driver you're using (Chrome, Firefox, etc.)

# Open the URL of the webpage
url="https://recetasdecocina.elmundo.es/tabla-calorias"
driver.get(url)

# Automatically scroll the page
scroll_pause_time = 2  # Pause between each scroll
screen_height = driver.execute_script("return window.screen.height;")  # Browser window height
i = 1
while True:
    # Scroll down
    driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
    i += 1
    time.sleep(scroll_pause_time)

    # Check if reaching the end of the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    if screen_height * i > scroll_height:
        break

# Fetch the data using BeautifulSoup after all data is loaded
soup = BeautifulSoup(driver.page_source, "html.parser")
# Process and save the data as needed

# Close the WebDriver session
driver.quit()

In [163]:
table = soup.find("table")
    
    # Extraer filas de la tabla
rows = table.find_all("tr")

    

In [164]:
import pandas as pd

In [165]:
tables = soup.find_all("table")

In [166]:
tablas=["verduras","Frutas","Frutos_secos","Lacteos","Carnes","Pescados","Dulces","Cereales","Legumbres","huevos","Pasteles","Bebidas","Aceites","Salsas"]


In [167]:
data = []
for i in range(len(tablas)):
    rows = tables[i].find_all("tr")
    

    for row in rows:
        cols = row.find_all("td")
        cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
        
        # Filtrar categorías que están en negrita y solo tienen una columna
        if len(cols) == 2:  
            data.append(cols)

In [168]:
df=pd.DataFrame(data)

In [169]:
indices_corte = df.index[df[1] == "CALORÍAS por cada 100 gramos"].tolist()
dfs_dict = {}

# Iterar sobre los índices y crear subconjuntos
for i in range(len(indices_corte) - 1):
    inicio = indices_corte[i]
    fin = indices_corte[i + 1] - 1  # -1 para evitar solapamiento
    
    nombre = tablas[i]  # Obtener el nombre correspondiente
    dfs_dict[nombre] = df.loc[inicio:fin, :]  # Crear el subconjunto

# Guardar cada DataFrame en un archivo CSV con su nombre
for nombre, sub_df in dfs_dict.items():
    #sub_df.to_csv(f"{nombre}.csv", index=False)
    print(f"✅ {nombre}.csv guardado con {len(sub_df)} filas.")

✅ verduras.csv guardado con 44 filas.
✅ Frutas.csv guardado con 37 filas.
✅ Frutos_secos.csv guardado con 8 filas.
✅ Lacteos.csv guardado con 39 filas.
✅ Carnes.csv guardado con 50 filas.
✅ Pescados.csv guardado con 37 filas.
✅ Dulces.csv guardado con 12 filas.
✅ Cereales.csv guardado con 22 filas.
✅ Legumbres.csv guardado con 4 filas.
✅ huevos.csv guardado con 5 filas.
✅ Pasteles.csv guardado con 12 filas.
✅ Bebidas.csv guardado con 33 filas.
✅ Aceites.csv guardado con 6 filas.


In [170]:
dfs_dict

{'verduras':                                0                             1
 0          VERDURAS Y HORTALIZAS  CALORÍAS por cada 100 gramos
 1               Aceitunas negras                           349
 2               Aceitunas verdes                           132
 3                        Acelgas                            33
 4                           Ajos                           169
 5                     Alcachofas                            64
 6                           Apio                            20
 7                      Berenjena                            29
 8                         Berros                            21
 9                        Brócoli                            31
 10                     Calabacín                            31
 11                      Calabaza                            24
 12                       Cebolla                            47
 13                Cebolla tierna                            39
 14       Champiñón y otras 

In [177]:
filas = []
for categoria, df in dfs_dict.items():
    # Eliminar la primera fila porque no es un alimento real
    df_limpio = df.iloc[1:].copy()

    # Renombrar columnas para mayor claridad
    df_limpio.columns = ["alimento", "kcal"]

    # Agregar la categoría a cada fila
    df_limpio["categoria"] = categoria

    # Convertir kcal a tipo numérico (puede estar como string)
    df_limpio["kcal"] = pd.to_numeric(df_limpio["kcal"], errors="coerce")

    # Guardar filas
    filas.append(df_limpio)

# Unir todas las categorías en un solo DataFrame
df_alimentos = pd.concat(filas, ignore_index=True)

df_alimentos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   alimento   296 non-null    object
 1   kcal       296 non-null    int64 
 2   categoria  296 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.1+ KB


In [98]:
rows = tables[0].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
verduras = pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
verduras.drop(0,inplace=True)
verduras["Calorías por 100g"]=verduras["Calorías por 100g"].astype(int)

In [101]:
rows = tables[1].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
frutas = pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
frutas.drop(0,inplace=True)
frutas["Calorías por 100g"]=frutas["Calorías por 100g"].astype(int)

In [104]:
rows = tables[2].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Frutos_secos = pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Frutos_secos.drop(0,inplace=True)
Frutos_secos["Calorías por 100g"]=Frutos_secos["Calorías por 100g"].astype(int)

In [105]:
rows = tables[3].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Lacteos= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Lacteos.drop(0,inplace=True)
Lacteos["Calorías por 100g"]=Lacteos["Calorías por 100g"].astype(int)

In [136]:
rows = tables[4].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Carnes= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Carnes.drop(0,inplace=True)
Carnes["Calorías por 100g"]=Carnes["Calorías por 100g"].astype(int)

In [137]:
rows = tables[5].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Pescados= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Pescados.drop(0,inplace=True)
Pescados["Calorías por 100g"]=Pescados["Calorías por 100g"].astype(int)

In [ ]:
rows = tables[6].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Dulces= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Dulces.drop(0,inplace=True)
Dulces["Calorías por 100g"]=Dulces["Calorías por 100g"].astype(int)

In [152]:
rows = tables[13].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Salsas= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Salsas.drop(0,inplace=True)
Salsas["Calorías por 100g"]=Salsas["Calorías por 100g"].astype(int)

In [150]:
rows = tables[12].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Aceites= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Aceites.drop(0,inplace=True)
Aceites["Calorías por 100g"]=Aceites["Calorías por 100g"].astype(int)

In [ ]:
rows = tables[11].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Bebidas= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Bebidas.drop(0,inplace=True)
Bebidas["Calorías por 100g"]=Bebidas["Calorías por 100g"].astype(int)

In [147]:
rows = tables[10].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Pasteles= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Pasteles.drop(0,inplace=True)
Pasteles["Calorías por 100g"]=Pasteles["Calorías por 100g"].astype(int)

In [145]:
rows = tables[9].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
huevos= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
huevos.drop(0,inplace=True)
huevos["Calorías por 100g"]=huevos["Calorías por 100g"].astype(int)

In [143]:
rows = tables[8].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Legumbres= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Legumbres.drop(0,inplace=True)
Legumbres["Calorías por 100g"]=Legumbres["Calorías por 100g"].astype(int)

In [141]:
rows = tables[7].find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]  # Extraer texto de cada celda
    
    # Filtrar categorías que están en negrita y solo tienen una columna
    if len(cols) == 2:  
        data.append(cols)
Cereales= pd.DataFrame(data, columns=["Alimento", "Calorías por 100g"])
Cereales.drop(0,inplace=True)
Cereales["Calorías por 100g"]=Cereales["Calorías por 100g"].astype(int)

-----------------------------

Dividimos los grupos de alimentos por categorias (según comida en la que debe estar y el macronutriente predominante)

In [178]:
df_alimentos["categoria"].unique()

array(['verduras', 'Frutas', 'Frutos_secos', 'Lacteos', 'Carnes',
       'Pescados', 'Dulces', 'Cereales', 'Legumbres', 'huevos',
       'Pasteles', 'Bebidas', 'Aceites'], dtype=object)

In [179]:
categorias_ref = pd.DataFrame({
    "categoria": [
        "verduras", "Frutas", "Frutos_secos", "Lacteos", "Carnes",
        "Pescados", "Dulces", "Cereales", "Legumbres", "huevos",
        "Pasteles", "Bebidas", "Aceites"
    ],
    "macronutriente": [
        "CH", "CH", "Grasa", "Proteína", "Proteína",
        "Proteína", "CH", "CH", "Proteína", "Proteína",
        "CH", "CH", "Grasa"
    ],
    "momento": [
        "Comida", "Desayuno", "Merienda", "Desayuno", "Comida",
        "Comida", "Desayuno", "Comida", "Comida", "Comida",
        "Desayuno", "Todo el día", "Comida"
    ]
})



In [180]:
df_alimentos_macros = df_alimentos.merge(categorias_ref, on="categoria", how="left")


In [181]:
df_alimentos_macros

,alimento,kcal,categoria,macronutriente,momento
0,Aceitunas negras,349,verduras,CH,Comida
1,Aceitunas verdes,132,verduras,CH,Comida
2,Acelgas,33,verduras,CH,Comida
3,Ajos,169,verduras,CH,Comida
4,Alcachofas,64,verduras,CH,Comida
...,...,...,...,...,...
291,Aceite de girasol,900,Aceites,Grasa,Comida
292,Aceite de oliva,900,Aceites,Grasa,Comida
293,Manteca,670,Aceites,Grasa,Comida
294,Mantequilla,752,Aceites,Grasa,Comida


In [2]:
import pandas as pd
import numpy as np

# Definir alimentos con su categoría, macronutriente y kcal por 100g
alimentos = [
    ("Arroz", "Cereales", "CH", 130),
    ("Pan integral", "Cereales", "CH", 250),
    ("Pasta", "Cereales", "CH", 150),
    ("Pollo", "Carnes", "Proteína", 165),
    ("Carne de res", "Carnes", "Proteína", 250),
    ("Pescado", "Carnes", "Proteína", 200),
    ("Huevo", "Huevos", "Proteína", 155),
    ("Leche", "Lácteos", "Proteína", 60),
    ("Yogur", "Lácteos", "Proteína", 80),
    ("Aceite de oliva", "Aceites", "Grasa", 900),
    ("Mantequilla", "Aceites", "Grasa", 752),
    ("Almendras", "Frutos secos", "Grasa", 576),
    ("Manzana", "Frutas", "CH", 52),
    ("Plátano", "Frutas", "CH", 89),
    ("Zanahoria", "Verduras", "CH", 35),
    ("Brócoli", "Verduras", "CH", 55)
]

# Posibles momentos de consumo
momentos = ["Desayuno", "Comida", "Cena"]

# Generar comidas sintéticas
np.random.seed(42)
num_comidas = 300
datos = []

for comida_id in range(1, num_comidas + 1):
    num_alimentos = np.random.randint(2, 5)  # Cada comida tendrá de 2 a 4 alimentos
    indices_seleccionados = np.random.choice(len(alimentos), num_alimentos, replace=False)
    momento = np.random.choice(momentos)
    
    comida = []
    kcal_total = 0

    for idx in indices_seleccionados:
        alimento = alimentos[idx]
        cantidad = np.random.randint(50, 200)  # Cantidades entre 50g y 200g
        kcal = (cantidad / 100) * alimento[3]  # Calcular kcal totales
        kcal_total += kcal

        comida.append([comida_id, alimento[0], cantidad, kcal, alimento[2], alimento[1], momento])

    # Guardar la comida generada
    datos.extend(comida)

# Crear DataFrame
df_comidas = pd.DataFrame(datos, columns=["comida_id", "alimento", "cantidad (g)", "kcal", "macronutriente", "categoría", "momento"])

# Calcular kcal totales por comida
df_kcal_totales = df_comidas.groupby("comida_id")["kcal"].sum().reset_index()
df_kcal_totales.rename(columns={"kcal": "kcal_totales"}, inplace=True)

# Unir las kcal totales con el dataset principal
df_comidas = df_comidas.merge(df_kcal_totales, on="comida_id")

# Mostrar las primeras filas
df_comidas.head()


,comida_id,alimento,cantidad (g),kcal,macronutriente,categoría,momento,kcal_totales
0,1,Arroz,51,66.30,CH,Cereales,Desayuno,681.25
1,1,Pan integral,137,342.50,CH,Cereales,Desayuno,681.25
2,1,Pescado,87,174.00,Proteína,Carnes,Desayuno,681.25
3,1,Brócoli,179,98.45,CH,Verduras,Desayuno,681.25
4,2,Pan integral,70,175.00,CH,Cereales,Cena,238.44


In [3]:
# Aumentar la cantidad de comidas generadas
num_comidas = 1000  # Más comidas para alcanzar más de 1000 filas
datos = []

for comida_id in range(1, num_comidas + 1):
    num_alimentos = np.random.randint(2, 5)  # Cada comida tendrá de 2 a 4 alimentos
    indices_seleccionados = np.random.choice(len(alimentos), num_alimentos, replace=False)
    momento = np.random.choice(momentos)

    comida = []
    kcal_total = 0

    for idx in indices_seleccionados:
        alimento = alimentos[idx]
        cantidad = np.random.randint(50, 200)  # Cantidades entre 50g y 200g
        kcal = (cantidad / 100) * alimento[3]  # Calcular kcal totales
        kcal_total += kcal

        comida.append([comida_id, alimento[0], cantidad, kcal, alimento[2], alimento[1], momento])

    # Guardar la comida generada
    datos.extend(comida)

# Crear DataFrame con más datos
df_comidas = pd.DataFrame(datos, columns=["comida_id", "alimento", "cantidad (g)", "kcal", "macronutriente", "categoría", "momento"])

# Calcular kcal totales por comida
df_kcal_totales = df_comidas.groupby("comida_id")["kcal"].sum().reset_index()
df_kcal_totales.rename(columns={"kcal": "kcal_totales"}, inplace=True)

# Unir las kcal totales con el dataset principal
df_comidas = df_comidas.merge(df_kcal_totales, on="comida_id")

# Verificar número de filas generadas
df_comidas.shape


(2990, 8)

In [6]:
df_comidas["alimento"].unique()

array(['Pasta', 'Manzana', 'Arroz', 'Leche', 'Huevo', 'Zanahoria',
       'Mantequilla', 'Pescado', 'Pollo', 'Carne de res',
       'Aceite de oliva', 'Brócoli', 'Almendras', 'Pan integral',
       'Plátano', 'Yogur'], dtype=object)